#WELCOME
In this Assignment, you'll use **Classifier** to predict the output. You are allowed to write/edit between the
```# START CODE HERE``` & ```# END CODE HERE``` only.<br>
Don't edit the test cells otherwise you'd be failed in the assignment.

Upload the ```utils.zip``` file before running the 1st cell.
[Demo](https://drive.google.com/file/d/1ebkQR6ikjGottmxa5KL83ZWdjTLvRxbm/view?usp=share_link)

**BEST OF LUCK**

**RUN THIS CODE TO UNIZP THE REQUIRED FILES**

In [ ]:
# !unzip -j /content/utils.zip -d.

# Dataset Loading and Pre-Processing

In [104]:
import pandas as pd
data = pd.read_csv("content/weather_classification_data.csv")

## Data Info
```data.info``` gives info about data-types and number of null objects.  
There are no null data, no need to remove any.

In [105]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13200 entries, 0 to 13199
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Temperature           13200 non-null  float64
 1   Humidity              13200 non-null  int64  
 2   Wind Speed            13200 non-null  float64
 3   Precipitation (%)     13200 non-null  float64
 4   Cloud Cover           13200 non-null  object 
 5   Atmospheric Pressure  13200 non-null  float64
 6   UV Index              13200 non-null  int64  
 7   Season                13200 non-null  object 
 8   Visibility (km)       13200 non-null  float64
 9   Location              13200 non-null  object 
 10  Weather Type          13200 non-null  object 
dtypes: float64(5), int64(2), object(4)
memory usage: 1.1+ MB


## Data Shuffling
Data Shuffling removes any bias present and creates uniform distribution.

In [106]:
from sklearn.utils import shuffle
data = shuffle(data)
data.head()

,Temperature,Humidity,Wind Speed,Precipitation (%),Cloud Cover,Atmospheric Pressure,UV Index,Season,Visibility (km),Location,Weather Type
11429,40.0,104,14.0,92.0,overcast,1019.13,0,Summer,10.5,coastal,Cloudy
10851,-2.0,91,11.5,87.0,partly cloudy,984.36,0,Winter,3.5,inland,Snowy
11680,16.0,53,11.5,31.0,partly cloudy,1015.88,3,Summer,6.5,coastal,Cloudy
4193,32.0,88,15.0,85.0,overcast,1010.10,0,Summer,4.5,coastal,Rainy
8912,3.0,87,7.5,77.0,overcast,990.74,0,Winter,1.5,mountain,Snowy


## Scaling and Labeling
Machine can't understand string so all the object datatypes need to be LABELED with integers.<br><br>
Why SCALING?<br>
You might have noticed one column contains data with large values (of order 1000) whereas some column contain small value data (of order 10). As a result, large value columns get more power/importance than others. To remove this disparity we scaled them to same order/values.

**CODE**

We need to find the columns with object datatypes.

HINT: Use ```data.select_dtypes().columns.tolist()```<br>Reference: [w3schools](https://www.w3schools.com/python/pandas/ref_df_select_dtypes.asp#:~:text=The%20select_dtypes()%20method%20returns,the%20specified%20dtype(s).&text=Note%3A%20You%20must%20specify%20at,you%20will%20get%20an%20error.)

In [107]:
# START CODE HERE
object_columns = data.select_dtypes(include='object').columns.tolist()
non_object_columns = data.select_dtypes(exclude='object').columns.tolist()
# END CODE HERE
print(f"Object Columns: {object_columns}\nNon Object Columns: {non_object_columns}")

Object Columns: ['Cloud Cover', 'Season', 'Location', 'Weather Type']
Non Object Columns: ['Temperature', 'Humidity', 'Wind Speed', 'Precipitation (%)', 'Atmospheric Pressure', 'UV Index', 'Visibility (km)']


**TEST** (DON'T EDIT THIS CELL)

In [108]:
# DON'T EDIT THIS CELL
if (object_columns == ['Cloud Cover', 'Season', 'Location', 'Weather Type'] and non_object_columns == ['Temperature', 'Humidity', 'Wind Speed', 'Precipitation (%)', 'Atmospheric Pressure', 'UV Index', 'Visibility (km)']):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed


**CODE**

We need to convert the object-type columns data into integral labels and scale the non-object-type columns data.

HINT: Use ```ColumnTransformer()```<br>Reference: [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), [ChatGPT](https://chatgpt.com/c/7f3c1b02-5da0-4c34-bdb2-d999dccb9c2f)

In [109]:
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

# START CODE HERE

column_transformer = ColumnTransformer(transformers=[
    ('oe', OrdinalEncoder(), object_columns),
    ('ss', StandardScaler(), non_object_columns)
], remainder='passthrough')
# Fit and transform the data
data_scaled_labeled = column_transformer.fit_transform(data)

# END CODE HERE

all_columns = object_columns + non_object_columns

# Create the new DataFrame
data_scaled_labeled = pd.DataFrame(data_scaled_labeled, columns=all_columns)

**TEST** (DON'T EDIT THIS CELL)

In [110]:
# DON'T EDIT THIS CELL

data_scaled_labeled_check = pd.read_csv("content/data_scaled_labeled_check.csv")
import numpy as np
data_scaled_labeled_values = np.sort(data_scaled_labeled.values,axis=0)
data_scaled_labeled_check_values = np.sort(data_scaled_labeled_check.values,axis=0)
if np.allclose(data_scaled_labeled_values, data_scaled_labeled_check_values, equal_nan=True):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed


## Data Splitting

**CODE**

We need to drop the Weather Type column from data_scaled_labeled to get the X and pick Weather Type column to get Y

Use 90-10 split of data, i.e. 90% for training, 10% for testing

In [111]:
X = data_scaled_labeled.drop(columns=['Weather Type'])
y = data_scaled_labeled['Weather Type']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = 0.9, random_state = 10)

**TEST** (DON'T EDIT THIS CELL)

In [112]:
# DON'T EDIT THIS CELL

if (len(X_train)==11880 and len(X_test)==1320):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

if (len(y_train)==11880 and len(y_test)==1320):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed
Test Passed


In [113]:
X_train.head()

,Cloud Cover,Season,Location,Temperature,Humidity,Wind Speed,Precipitation (%),Atmospheric Pressure,UV Index,Visibility (km)
7017,2.0,3.0,2.0,0.337774,0.162883,0.748042,1.044147,-0.383833,-0.779410,-1.027154
12278,2.0,1.0,0.0,-0.352450,0.559050,0.820417,1.263271,0.262435,-0.520104,-0.137308
2859,3.0,3.0,1.0,0.280255,0.261924,-0.627087,0.605898,0.158666,1.813646,4.311920
5495,2.0,1.0,2.0,-0.294931,0.757134,0.458541,0.073739,-0.112315,-0.260799,-0.137308
8469,2.0,1.0,2.0,-0.179894,-0.381848,-0.337587,-0.270600,0.247918,-0.779410,0.455922


# Training and Testing
Use any suitable classifier to fit the training data and check the accuracy on test data

Don't forget to import that library

**CODE**

ACCURACY should be more than 90%,

1. Fit the model to X_train and y_train
2. Find the model score on X_test and y_test

In [114]:
# START CODE HERE

from sklearn.svm import SVC

model = SVC(C=5)
model.fit(X_train,y_train)
score = model.score(X_test,y_test)

# END CODE HERE

print("Accuracy:", score*100)

Accuracy: 90.68181818181819


**TEST** (DON'T EDIT THIS CELL)

In [115]:
# DON'T EDIT THIS CELL

if (score>0.90):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed


# Predicting Output (Here, Weather Type) (Optional)

In [116]:
input = X_test
input

,Cloud Cover,Season,Location,Temperature,Humidity,Wind Speed,Precipitation (%),Atmospheric Pressure,UV Index,Visibility (km)
11923,2.0,3.0,1.0,-0.870118,-0.332327,-1.133714,1.138057,-0.168500,-1.038715,-0.582231
2331,2.0,3.0,2.0,-1.330267,-1.223703,-1.423215,0.637201,-3.872176,0.257813,0.010999
217,2.0,1.0,1.0,-0.007338,1.301864,0.458541,0.887629,-0.063119,-0.520104,-0.433923
1760,2.0,1.0,2.0,-0.582524,-0.134243,-1.061339,-0.708848,4.387906,2.332257,1.197460
3963,0.0,2.0,1.0,0.452811,-1.025620,-0.482337,-1.241007,0.632883,1.035729,0.752538
...,...,...,...,...,...,...,...,...,...,...
8256,2.0,3.0,1.0,-1.560341,-0.332327,0.892792,0.355470,-0.341358,-0.779410,-1.027154
5115,3.0,3.0,1.0,0.510330,0.162883,0.024290,-0.990580,-0.053172,-0.001493,0.752538
2862,2.0,2.0,2.0,0.625367,0.509529,-0.699463,0.981540,-0.390285,-0.520104,-0.285616
4439,2.0,3.0,1.0,-1.445304,1.252343,-1.061339,1.231967,-0.446740,-0.779410,-1.175461


In [117]:
y = model.predict(input)
counts=0
print(y)
# some = y_test.to_numpy
# for i in range(len(y)):
#     if y[i]==some[i]:
#         counts+=1

[2. 0. 1. ... 1. 2. 0.]


Here, the outputs are in numbers. We can perform inverse column transfer for getiing the string values.


In [120]:
from sklearn.ensemble import RandomForestClassifier
model2 = RandomForestClassifier(n_estimators=25, criterion='gini')
model2.fit(X_train, y_train)
score = model2.score(X_test,y_test)

# END CODE HERE

print("Accuracy:", score*100)

Accuracy: 92.95454545454545
